Mount the drive 

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Necessary imports

In [0]:
import numpy as np
import cv2
import pandas as pd
import random
import os

In [0]:
!mkdir data
os.chdir('data/')

In [0]:
ls   

change the link to the link of your fer2013.csv file link

In [0]:
ferlink =  '../gdrive/My Drive/privateaiproject/fer2013.csv'

In [0]:
os.getcwd()

'/content/data'

In [0]:
# os.chdir('../')

In [0]:
curdir = os.getcwd()


A function to create a directory for each class and move the images into those folders

In [0]:
# curdir = os.path.abspath(os.path.dirname(__file__))
def gen_record(csvfile,channel):
    data = pd.read_csv(csvfile,delimiter=',',dtype='a')
    labels = np.array(data['emotion'],np.float)
    # print(labels,'\n',data['emotion'])
        
    imagebuffer = np.array(data['pixels'])
    images = np.array([np.fromstring(image,np.uint8,sep=' ') for image in imagebuffer])
    del imagebuffer
    num_shape = int(np.sqrt(images.shape[-1]))
    images.shape = (images.shape[0],num_shape,num_shape)
    # img=images[0];cv2.imshow('test',img);cv2.waitKey(0);cv2.destroyAllWindow();exit()
    dirs = set(data['Usage'])
    subdirs = set(labels)
    class_dir = {}
    for dr in dirs:
        dest = os.path.join(curdir,dr)
        class_dir[dr] = dest
        if not os.path.exists(dest):
            os.mkdir(dest)
            
    data = zip(labels,images,data['Usage'])
    
    for d in data:
        destdir = os.path.join(class_dir[d[-1]],str(int(d[0])))
        if not os.path.exists(destdir):
            os.mkdir(destdir)
        img = d[1]
        filepath = unique_name(destdir,d[-1])
        print('[^_^] Write image to %s' % filepath)
        if not filepath:
            continue
        sig = cv2.imwrite(filepath,img)
        if not sig:
            print('Error')
            exit(-1)


def unique_name(pardir,prefix,suffix='jpg'):
    filename = '{0}_{1}.{2}'.format(prefix,random.randint(1,10**8),suffix)
    filepath = os.path.join(pardir,filename)
    if not os.path.exists(filepath):
        return filepath
    unique_name(pardir,prefix,suffix)
    




Call the function on the link

In [0]:

# filename = fer2013link
# filename = os.path.join(curdir,filename)
gen_record(ferlink,1)
    


Output hidden; open in https://colab.research.google.com to view.

install pysyft

In [0]:
!pip install syft

     |████████████████████████████████| 256kB 4.6MB/s 
     |████████████████████████████████| 256kB 42.0MB/s 
     |████████████████████████████████| 2.1MB 36.3MB/s 
     |████████████████████████████████| 460kB 38.4MB/s 
     |████████████████████████████████| 204kB 38.2MB/s 
     |████████████████████████████████| 81kB 24.4MB/s 
     |████████████████████████████████| 389kB 39.0MB/s 
     |████████████████████████████████| 51kB 17.5MB/s 
     |████████████████████████████████| 266kB 40.4MB/s 
     |████████████████████████████████| 122kB 43.0MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067070 sha256=5a8f43e8295273f871c041417b975d70a1dad74ea2cb5b705702197d5e4ddd0a
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=e941087b242f60df2267c188ba06f0b083adc402e01778ddad0d5248e3d47f88
  Stored in di

import torch libraries 

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import pysyft and create three workers and a crypto provider

In [0]:
import syft as sy  
hook = sy.TorchHook(torch) 
client = sy.VirtualWorker(hook, id="client") 
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider") 


W0820 01:32:14.940291 140462754883456 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0820 01:32:14.961719 140462754883456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



some basic args

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 256
        self.test_batch_size = 200
        self.epochs = 1
        self.lr = 0.0001 # learning rate
        self.log_interval = 100

args = Arguments()


load the images transform them and create data loaders 

In [0]:

normalize = transforms.Compose([transforms.Resize((48,48)),
                            transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor(), 
                               transforms.Normalize((0.1307,), (0.3081,))])
train_dataset =datasets.ImageFolder('Training',   transform=normalize)
test_dataset = datasets.ImageFolder('PublicTest', transform=normalize)


train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    shuffle=True)

In [0]:
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=args.test_batch_size,
    shuffle=True)

# Convert to integers and privately share the dataset
private_test_loader = []
for data, target in test_loader:
    private_test_loader.append((
        data.fix_prec().share(alice, bob, crypto_provider=crypto_provider),
        target.fix_prec().share(alice, bob, crypto_provider=crypto_provider)
    ))


use  a simple neural net for baseline

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

it had high bias so added more layers

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 500)
        self.fc4 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)        
        x = self.fc2(x)
        x = F.relu(x)        
        x = self.fc3(x)
        x = F.relu(x)
#         x = F.dropout(x,p=0.2)
        x = self.fc4(x)
        return x

started overfitting so added dropout

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 500)
        self.fc4 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)        
        x = self.fc2(x)
        x = F.relu(x)
        x = F.dropout(x,p=0.5)
        x = self.fc3(x)
        x = F.relu(x)
        x = F.dropout(x,p=0.5)

        x = self.fc4(x)
        return x

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
#         self.fc1 = nn.Linear(784, 1000)
#         self.fc2 = nn.Linear(1000, 500)
#         self.fc3 = nn.Linear(500, 500)
#         self.fc4 = nn.Linear(500, 10)

        self.cnn1a = nn.Conv2d(1,32, 3, padding=2, dilation = 2)
        self.cnn1b = nn.Conv2d(32,32, 3, padding=2)
        self.cnn1_maxpool = nn.MaxPool2d(2)

        # Second Sparse CNN layer
        self.cnn2a = nn.Conv2d(32, 64, 3, padding=2, dilation = 2)
        self.cnn2b = nn.Conv2d(64,64, 3, padding=2)
        self.cnn2_maxpool = nn.MaxPool2d(2)

        # Third Sparse CNN layer
        self.cnn3a = nn.Conv2d(64, 96, 3, padding=2, dilation = 2)
        self.cnn3b = nn.Conv2d(96,96, 3)
        self.cnn3_maxpool = nn.MaxPool2d(2)



        # Fourth Sparse CNN layer
        self.cnn4a = nn.Conv2d(96, 128, 3, padding=2, dilation = 2)
        self.cnn4b = nn.Conv2d(128,128, 3)
        self.cnn4_maxpool = nn.MaxPool2d(2)


#         self.flatten", Flatten()

        # Sparse Linear layer
        self.linear = nn.Linear(128, 64)

        # Classifier
        self.output = nn.Linear(64, 7)


        
    def forward(self, x):
#         x = x.view(-1, 784)
        x = self.cnn1a(x)
        x = self.cnn1b(x)
        x = self.cnn1_maxpool(x)
        x = self.cnn2a(x)
        x = self.cnn2b(x)
        x = self.cnn2_maxpool(x)
        x = self.cnn3a(x)
        x = self.cnn3b(x)
        x = self.cnn3_maxpool(x)
        x = self.cnn4a(x)
        x = self.cnn4b(x)
        x = self.cnn4_maxpool(x)        
        x = x.view(x.size()[0], -1)
        
        
#         x = F.relu(x)        
        x = self.linear(x)
        x = F.relu(x)
#         x = F.dropout(x,p=0.5)
        x = self.output(x)
#         x = F.relu(x)
#         x = F.dropout(x,p=0.5)

#         x = self.fc4(x)
        return x

In [0]:
def train(args, model, train_loader, test_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        print("tr start")
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
#     if batch_idx % args.log_interval == 0:
    print('Train Epoch: {} '.format(epoch))
#     if batch_idx % (args.log_interval * 8) == 0:
#                     tr_loss = []
    te_loss = []
    n_correct_priv=0
#         model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
        for data, target in test_loader:
            outputt = model(data)
            outputt = F.log_softmax(outputt, dim=1)
            losste = F.nll_loss(outputt, target)
            te_loss.append(losste.numpy())
            
            pred = outputt.argmax(dim=1) 
            n_correct_priv += pred.eq(target.view_as(pred)).sum()
            n_total += args.test_batch_size
            
#             n_correct = n_correct_priv.copy().get().float_precision().long().item()
    
            print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
                n_correct_priv, n_total,
                100. * n_correct / n_total))
            
            
        print('train loss ', loss)
        print('test loss ', np.average(np.array(te_loss)))


model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, train_loader, test_loader, optimizer, epoch)


In [0]:
model.fix_precision().share(alice, bob, crypto_provider=crypto_provider)

In [0]:
def test(args, model, test_loader):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            pred = output.argmax(dim=1) 
            n_correct_priv += pred.eq(target.view_as(pred)).sum()
            n_total += args.test_batch_size
            
            n_correct = n_correct_priv.copy().get().float_precision().long().item()
    
            print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
                n_correct, n_total,
                100. * n_correct / n_total))

test(args, model, private_test_loader)